In [29]:
# Import necessary libraries
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim import corpora, models
from gensim.test.utils import datapath
import gensim
import pyLDAvis
import pyLDAvis.gensim_models

# Preprocessing

In [11]:
# Function to load data from folder path
def load_data(folder_path):
    articles = []
    
	# Iterate through all files in the folder path
    for file in os.listdir(folder_path):
        with open(os.path.join(folder_path, file), "r", encoding="latin") as f:
            content = f.read()
            articles.append(content)
    return articles

In [12]:
# Function to preprocess the loaded data
def preprocess_data(articles):
    cleaned_articles = []
     # Iterate through all articles in the input list   

    for article in articles:
	# Remove excessive whitespace characters and convert to lowercase
        text = re.sub(r"\s+", " ", article)
        text = re.sub(r"[^a-zA-Z\s]", "", text).lower()
	# Tokenize text into words
        tokens = word_tokenize(text)
	# Remove stop words (common words with little meaning such as "the" or "a")
        stop_words = set(stopwords.words("english"))
        custom_stopwords = {"mr", "said", "u"}  # Add any additional stopwords here
        stop_words = stop_words.union(custom_stopwords)
	# Remove any remaining short words and lemmatize 
        tokens = [token for token in tokens if token not in stop_words]
        tokens = [token for token in tokens if len(token) > 1 or token in {"i", "a"}]
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        tokens = [token for token in tokens if token != "u"]
        cleaned_articles.append(tokens)
    return cleaned_articles

In [13]:
# Function to create a corpus and dictionary from the preprocessed data
def create_corpus_dictionary(cleaned_articles):
    dictionary = corpora.Dictionary(cleaned_articles)
    corpus = [dictionary.doc2bow(article) for article in cleaned_articles]
    return corpus, dictionary

In [14]:
# Run preprocessing functions
folder_path = "Articles/"
articles = load_data(folder_path)
cleaned_articles = preprocess_data(articles)
corpus, dictionary = create_corpus_dictionary(cleaned_articles)

# Topic Models

In [15]:
# Function to train an LDA model
def train_lda(corpus, dictionary, num_topics, passes=15):
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=passes)
    return lda_model

In [16]:
# Function to print and plot the topics from an LDA model
def plot_topics(lda_model):
    num_topics = lda_model.num_topics
    topic_words = lda_model.print_topics(num_words=10)
    
    for topic in range(num_topics):
        print(f"Topic {topic}:")
        print(topic_words[topic])

In [17]:
# Function to run the LDA training and save the results to a file
def run_and_save_lda(corpus, dictionary, num_topics, passes, model_number, output_file):
    lda_model = train_lda(corpus, dictionary, num_topics, passes)

    # save the 4 models to Models/ folder
    temp_file = 'Models/lda_model_' + str(model_number+1)
    lda_model.save(temp_file)

    # Save the topic words to output files
    topic_words = lda_model.print_topics(num_words=10)
    with open(output_file, "w") as f:
        for topic in range(num_topics):
            f.write(f"Topic {topic}:\n")
            f.write(str(topic_words[topic]) + "\n\n")

In [18]:
# Define different parameters to build the LDA model
# use different topic and pass count and discuss differences in report
params = [{"num_topics": 5, "passes": 10},{"num_topics": 10, "passes": 10},{"num_topics": 5, "passes": 20},{"num_topics": 10, "passes": 20}]


# Iterating through the parameters and building different LDA models
for i, p in enumerate(params):
    output_file = f"output_{i + 1}.txt"
    run_and_save_lda(corpus, dictionary,  p["num_topics"], p["passes"], i, output_file)

# Evaluation

In [19]:
# Load models from Models folder
model_1 = models.ldamodel.LdaModel.load("Models/lda_model_1")
model_2 = models.ldamodel.LdaModel.load("Models/lda_model_2")
model_3 = models.ldamodel.LdaModel.load("Models/lda_model_3")
model_4 = models.ldamodel.LdaModel.load("Models/lda_model_4")
models_list = [model_1, model_2, model_3, model_4]

In [20]:
# Compute Perplexity of all 4 models
for m in models_list:
    print('\nPerplexity: ', m.log_perplexity(corpus))


Perplexity:  -8.493143757989483

Perplexity:  -8.502332575847719

Perplexity:  -8.478354714100588

Perplexity:  -8.504284328414839


# Visualization

In [30]:
# Enable Python Notebook display of pyLDAvis
pyLDAvis.enable_notebook

# Model 1 visualization
vis = pyLDAvis.gensim_models.prepare(model_1, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.018137  0.004336       1        1  28.781117
2     -0.022329 -0.032460       2        1  27.814665
3     -0.013919 -0.027676       3        1  25.904359
1      0.062427  0.012030       4        1  10.281838
4     -0.044316  0.043769       5        1   7.218020, topic_info=          Term          Freq         Total Category  logprob  loglift
2251     trump   7446.000000   7446.000000  Default  30.0000  30.0000
1119   islamic   6047.000000   6047.000000  Default  29.0000  29.0000
2079     state  10059.000000  10059.000000  Default  28.0000  28.0000
40136    gigot   1032.000000   1032.000000  Default  27.0000  27.0000
189     attack   3558.000000   3558.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
867      force    171.212357   3506.015695   Topic5  -6.1031  -0.3907
2293    united    166.634334   3342.016641   Topic5  -6.1302  -0.3699
1201      last    148.261502   2182.789184   Topic5  -6.2470  -0.0608
513      could    147.788770   2168.706586   Topic5  -6.2502  -0.0575
1115       isi    147.817830   2733.673248   Topic5  -6.2500  -0.2888

[473 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
0          1  0.226015       abadi
0          2  0.115764       abadi
0          3  0.176402       abadi
0          4  0.468568       abadi
0          5  0.011025       abadi
...      ...       ...         ...
12533      1  0.068071  zuckerberg
12533      2  0.907615  zuckerberg
12533      3  0.007563  zuckerberg
12533      4  0.007563  zuckerberg
35866      5  0.825080        zway

[969 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 2, 5])

In [31]:
# Model 2 visualization
vis = pyLDAvis.gensim_models.prepare(model_2, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.118444  0.014396       1        1  38.185117
0     -0.149858 -0.024588       2        1  26.163510
6     -0.128888  0.003606       3        1  22.164578
9     -0.127719 -0.058651       4        1   6.741946
1     -0.105742  0.073632       5        1   6.516729
8      0.082369 -0.007369       6        1   0.227438
3      0.095119  0.009259       7        1   0.000170
4      0.140207 -0.007187       8        1   0.000170
2      0.148076 -0.001495       9        1   0.000170
5      0.164880 -0.001603      10        1   0.000170, topic_info=          Term          Freq         Total Category  logprob  loglift
2079     state  10228.000000  10228.000000  Default  30.0000  30.0000
2251     trump   7585.000000   7585.000000  Default  29.0000  29.0000
1418       new   6127.000000   6127.000000  Default  28.0000  28.0000
1119   islamic   6193.000000   6193.000000  Default  27.0000  27.0000
2206      time   5802.000000   5802.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
1115       isi      0.000615   2785.706052  Topic10  -7.9865  -2.0442
1338  militant      0.000508   1580.468756  Topic10  -8.1765  -1.6674
2384      week      0.000556   2071.775812  Topic10  -8.0878  -1.8494
1482  official      0.000612   3362.774478  Topic10  -7.9911  -2.2370
2432     would      0.000594   3846.777459  Topic10  -8.0215  -2.4019

[822 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
40021      4  0.970361   abdeslams
17089      1  0.898635       abedi
17089      2  0.093500       abedi
17089      4  0.005194       abedi
8          6  0.893817       abort
...      ...       ...         ...
33837      1  0.984606      zarrab
17077      3  0.967106    zucchino
12533      1  0.902211  zuckerberg
12533      2  0.073351  zuckerberg
12533      3  0.022005  zuckerberg

[1231 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 7, 10, 2, 9, 4, 5, 3, 6])